In [12]:
from azureml.core import Dataset, Run, Workspace

# run = Run.get_context()
# workspace = run.experiment.workspace

workspace = Workspace.from_config()

dataset_name = 'dataset2'

# Get a dataset by name
ds = Dataset.get_by_name(workspace=workspace, name=dataset_name)

# Load a TabularDataset into pandas DataFrame
df = ds.to_pandas_dataframe()


2022-10-06:17:14:32,873 INFO     [workspace.py:291] Found the config file in: /config.json
2022-10-06:17:14:32,910 INFO     [_universal.py:476] Request URL: 'https://management.azure.com/subscriptions?api-version=REDACTED'
Request method: 'GET'
Request headers:
    'Accept': 'application/json'
    'x-ms-client-request-id': '586b1418-459a-11ed-83c4-f5a207509f78'
    'User-Agent': 'azsdk-python-azure-mgmt-resource/21.1.0 Python/3.8.5 (Linux-5.15.0-1017-azure-x86_64-with-glibc2.10)'
    'Authorization': 'REDACTED'
No body was attached to the request
2022-10-06:17:14:32,965 INFO     [_universal.py:504] Response status: 200
Response headers:
    'Cache-Control': 'no-cache'
    'Pragma': 'no-cache'
    'Content-Type': 'application/json; charset=utf-8'
    'Expires': '-1'
    'x-ms-ratelimit-remaining-tenant-reads': '11997'
    'x-ms-request-id': 'e8bb8d7e-5063-40bd-900a-e0b2bfcc218a'
    'x-ms-correlation-request-id': 'REDACTED'
    'x-ms-routing-request-id': 'REDACTED'
    'Strict-Transport

In [13]:
df

,Order Id,first_name,last_name,gender,Date,Product,City,Quantity,price
0,1,Barrie,Gamage,Male,2022-07-09,Cmore Ice Ski,FerreÃ±afe,3,€ 450.00
1,2,Fenelia,Stoppard,Female,2022-07-11,Cmore Ice Ski,DamÄvand,2,€ 450.00
2,3,Krystle,Manvelle,Female,2022-06-24,Cmore Ice Ski,Yingzai,1,€ 450.00
3,4,Britt,Zapater,Female,2022-07-03,Cmore Ice Ski,Liren,1,€ 450.00
4,5,Britni,Gear,Female,2022-08-24,Cmore Ice Ski,San Antonio,1,€ 450.00
...,...,...,...,...,...,...,...,...,...
495,496,Rebecca,Marle,Female,2022-09-17,Cmore Ice Ski,MaturÃ­n,1,€ 850.00
496,497,Mabelle,Mandry,Female,2021-10-26,Cmore Ice Ski,Luci,3,€ 850.00
497,498,Suzann,Le Teve,Female,2022-04-16,Cmore Ice Ski,Ciakar,1,€ 850.00
498,499,Dory,Shrieve,Non-binary,2021-12-05,Cmore Ice Ski,Saint-Ouen,3,€ 850.00


In [14]:
df.describe

<bound method NDFrame.describe of      Order Id first_name  last_name      gender       Date        Product  \
0           1     Barrie     Gamage        Male 2022-07-09  Cmore Ice Ski   
1           2    Fenelia   Stoppard      Female 2022-07-11  Cmore Ice Ski   
2           3    Krystle   Manvelle      Female 2022-06-24  Cmore Ice Ski   
3           4      Britt    Zapater      Female 2022-07-03  Cmore Ice Ski   
4           5     Britni       Gear      Female 2022-08-24  Cmore Ice Ski   
..        ...        ...        ...         ...        ...            ...   
495       496    Rebecca      Marle      Female 2022-09-17  Cmore Ice Ski   
496       497    Mabelle     Mandry      Female 2021-10-26  Cmore Ice Ski   
497       498     Suzann    Le Teve      Female 2022-04-16  Cmore Ice Ski   
498       499       Dory    Shrieve  Non-binary 2021-12-05  Cmore Ice Ski   
499       500       Hali  Stansbury      Female 2021-12-12  Cmore Ice Ski   

            City  Quantity       price  


In [15]:
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(df, test_size=0.2, random_state=42)

In [16]:
import logging

automl_settings = {
    "iteration_timeout_minutes": 10,
    "experiment_timeout_hours": 0.3,
    "enable_early_stopping": True,
    "primary_metric": 'spearman_correlation',
    "featurization": 'auto',
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}


In [17]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='regression',
                             debug_log='automated_ml_errors.log',
                             training_data=x_train,
                             label_column_name="Quantity",
                             **automl_settings)


In [18]:
from azureml.core.experiment import Experiment
experiment = Experiment(workspace, "Tutorial-adaptation-regression")
local_run = experiment.submit(automl_config, show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
Tutorial-adaptation-regression,AutoML_b9848af2-b43d-4b07-9da7-aa90692d7ea0,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Running in the active local environment.
No run_configuration provided, running on local with default configuration
Running in the active local environment.
Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**************************************************************

2022-10-06:17:35:45,754 INFO     [explanation_client.py:334] Using default datastore for uploads


In [19]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [20]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)


Run(Experiment: Tutorial-adaptation-regression,
Id: AutoML_b9848af2-b43d-4b07-9da7-aa90692d7ea0_31,
Type: None,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='regression', working_dir='/mnt/batch/ta...
                                             PreFittedSoftVotingRegressor(estimators=[('25', Pipeline(memory=None, steps=[('sparsenormalizer', Normalizer(copy=True, norm='l2')), ('decisiontreeregressor', DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None, max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=0.5, min_samples_split=0.04427451843

In [22]:
y_test = x_test.pop("Quantity")

y_predict = fitted_model.predict(x_test)
print(y_predict[:10])


[1.86551224 1.84379001 1.74674805 1.78410185 1.73345668 1.78601224
 1.79879001 1.77834557 1.86501224 1.86890112]
